In [1]:
spark

In [2]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

### 1. Criar um dataframe para ler o arquivo no HDFS /user/<nome/data/juros_selic/juros_selic

In [3]:
# listando o arquivo no HDFS

!hdfs dfs -ls /user/feliciani/data/juros_selic

Found 3 items
-rw-r--r--   3 root supergroup       8348 2021-06-23 18:23 /user/feliciani/data/juros_selic/juros_selic
-rw-r--r--   3 root supergroup      14621 2021-06-23 18:23 /user/feliciani/data/juros_selic/juros_selic.json
-rw-r--r--   3 root supergroup      13147 2021-06-23 18:23 /user/feliciani/data/juros_selic/juros_selic.wsdl


In [4]:
# Conteúdo do Arquivo juros_selic

!hdfs dfs -cat /user/feliciani/data/juros_selic/juros_selic

# tem os campos data e valor, ambos com aspas e separados com ponto e vírgula

In [16]:
from pyspark.sql.functions import *

In [5]:
juros = spark.read.csv("/user/feliciani/data/juros_selic/juros_selic", sep=";", header="true")

# informei que o separador é ponto e vírgula e haverá cabeçalho

In [7]:
juros.printSchema()
juros.show(5)

# já está estruturado e tipado.

root
 |-- data: string (nullable = true)
 |-- valor: string (nullable = true)

+----------+-----+
|      data|valor|
+----------+-----+
|01/06/1986| 1,27|
|01/07/1986| 1,95|
|01/08/1986| 2,57|
|01/09/1986| 2,94|
|01/10/1986| 1,96|
+----------+-----+
only showing top 5 rows



### 2. Alterar o formato do campo data para “MM/dd/yyyy”

In [8]:
juros.show(10)

# está como dia, mês, ano

+----------+-----+
|      data|valor|
+----------+-----+
|01/06/1986| 1,27|
|01/07/1986| 1,95|
|01/08/1986| 2,57|
|01/09/1986| 2,94|
|01/10/1986| 1,96|
|01/11/1986| 2,37|
|01/12/1986| 5,47|
|01/01/1987|11,00|
|01/02/1987|19,61|
|01/03/1987|11,95|
+----------+-----+
only showing top 10 rows



### withColumn - Trabalhando com timestamp
### converte coluna para timestamp
### Altera formato de coluna de timestamp

In [17]:
# utilizando o withColumn e tranformando primeiro para unix_timestamp

# unix_timestamp(col(“<ColString>"), “<FormatoAtual>"),from_unixtime(<ColTimestamp>, "<FormatoConversão>"))

juros_americano_unix = juros.withColumn("data", unix_timestamp(col("data"),"dd/MM/yyyy"))

juros_americano = juros_americano_unix.withColumn("data", from_unixtime("data", "MM/dd/yyyy")).show(10)

# PODERIA SER ASSIM:
# juros_americano = juros.withColumn("data", from_unixtime(unix_timestamp(col("data"),"dd/MM/yyyy"), "MM/dd/yyyy"))
# juros_americano.show(10)

+----------+-----+
|      data|valor|
+----------+-----+
|06/01/1986| 1,27|
|07/01/1986| 1,95|
|08/01/1986| 2,57|
|09/01/1986| 2,94|
|10/01/1986| 1,96|
|11/01/1986| 2,37|
|12/01/1986| 5,47|
|01/01/1987|11,00|
|02/01/1987|19,61|
|03/01/1987|11,95|
+----------+-----+
only showing top 10 rows



### 3. Com uso da função from_unixtime crie o campo “ano_unix”, com a informação do ano do campo data

In [19]:
juros_unixtime = juros.withColumn("ano_unix", from_unixtime(unix_timestamp(col("data"),"dd/MM/yyyy"),"yyyy"))
juros_unixtime.show(5)

# Criou uma coluna ano_unix

+----------+-----+--------+
|      data|valor|ano_unix|
+----------+-----+--------+
|01/06/1986| 1,27|    1986|
|01/07/1986| 1,95|    1986|
|01/08/1986| 2,57|    1986|
|01/09/1986| 2,94|    1986|
|01/10/1986| 1,96|    1986|
+----------+-----+--------+
only showing top 5 rows



### 4. Com uso de substring crie o campo “ano_str”, com a informação do ano do campo data
### Extrai os dados de uma coluna de acordo com sua posição


In [21]:
# especificar qual coluna será utilizada ("data")
# Vai ler na sétima posição
# utilizará 4 posições

# <dataframe>.withColumn(“<nomeColuna>", substring(“<Coluna>”, <posicaoInicial>, <tamanho>)

juros_substring = juros_unixtime.withColumn("ano_str", substring(col("data"),7,4))
juros_substring.show(5)

+----------+-----+--------+-------+
|      data|valor|ano_unix|ano_str|
+----------+-----+--------+-------+
|01/06/1986| 1,27|    1986|   1986|
|01/07/1986| 1,95|    1986|   1986|
|01/08/1986| 2,57|    1986|   1986|
|01/09/1986| 2,94|    1986|   1986|
|01/10/1986| 1,96|    1986|   1986|
+----------+-----+--------+-------+
only showing top 5 rows



### 5. Com uso da função split crie o campo “ano_str”, com a informação do ano do campo data
### split - criar um array de acordo com um delimitar


In [27]:
# especificar qual coluna será separada ("data")

# <dataframe>.withColumn(“<nomeColuna>", split(“<Coluna>”, ”<delimitador” ))

juros_split = juros_substring.withColumn("ano_split", split(col("data"),"/"))
juros_split.show(5)

+----------+-----+--------+-------+--------------+
|      data|valor|ano_unix|ano_str|     ano_split|
+----------+-----+--------+-------+--------------+
|01/06/1986| 1,27|    1986|   1986|[01, 06, 1986]|
|01/07/1986| 1,95|    1986|   1986|[01, 07, 1986]|
|01/08/1986| 2,57|    1986|   1986|[01, 08, 1986]|
|01/09/1986| 2,94|    1986|   1986|[01, 09, 1986]|
|01/10/1986| 1,96|    1986|   1986|[01, 10, 1986]|
+----------+-----+--------+-------+--------------+
only showing top 5 rows



In [28]:
juros_split = juros_substring.withColumn("ano_split", split(col("data"),"/").getItem(2))
juros_split.show(5)

+----------+-----+--------+-------+---------+
|      data|valor|ano_unix|ano_str|ano_split|
+----------+-----+--------+-------+---------+
|01/06/1986| 1,27|    1986|   1986|     1986|
|01/07/1986| 1,95|    1986|   1986|     1986|
|01/08/1986| 2,57|    1986|   1986|     1986|
|01/09/1986| 2,94|    1986|   1986|     1986|
|01/10/1986| 1,96|    1986|   1986|     1986|
+----------+-----+--------+-------+---------+
only showing top 5 rows



### 6. Salvar no hdfs /user/rodrigo/juros_selic_americano no formato CSV, incluindo o cabeçalho

In [30]:
juros_split.write.csv("/user/feliciani/juros_selic", header="true")

In [31]:
!hdfs dfs -ls /user/feliciani/juros_selic

Found 2 items
-rw-r--r--   2 root supergroup          0 2021-06-29 18:28 /user/feliciani/juros_selic/_SUCCESS
-rw-r--r--   2 root supergroup      13086 2021-06-29 18:28 /user/feliciani/juros_selic/part-00000-d4b25d70-03c7-48b0-9b46-61662fbeb9fb-c000.csv


In [32]:
!hdfs dfs -cat /user/feliciani/juros_selic/part-00000-d4b25d70-03c7-48b0-9b46-61662fbeb9fb-c000.csv

data,valor,ano_unix,ano_str,ano_split
01/06/1986,"1,27",1986,1986,1986
01/07/1986,"1,95",1986,1986,1986
01/08/1986,"2,57",1986,1986,1986
01/09/1986,"2,94",1986,1986,1986
01/10/1986,"1,96",1986,1986,1986
01/11/1986,"2,37",1986,1986,1986
01/12/1986,"5,47",1986,1986,1986
01/01/1987,"11,00",1987,1987,1987
01/02/1987,"19,61",1987,1987,1987
01/03/1987,"11,95",1987,1987,1987
01/04/1987,"15,30",1987,1987,1987
01/05/1987,"24,63",1987,1987,1987
01/06/1987,"18,02",1987,1987,1987
01/07/1987,"8,91",1987,1987,1987
01/08/1987,"8,09",1987,1987,1987
01/09/1987,"7,99",1987,1987,1987
01/10/1987,"9,45",1987,1987,1987
01/11/1987,"12,92",1987,1987,1987
01/12/1987,"14,38",1987,1987,1987
01/01/1988,"16,78",1988,1988,1988
01/02/1988,"18,35",1988,1988,1988
01/03/1988,"16,59",1988,1988,1988
01/04/1988,"20,25",1988,1988,1988
01/05/1988,"18,65",1988,1988,1988
01/06/1988,"20,17",1988,1988,1988
01/07/1988,"24,69",1988,1988,1988
01/08/1988,"22,63",1988,1988,1988
01/09/1988,"26,25",1988,1988,1988
01/10/1988,"29,79",19